In [33]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
import numpy as np
from pathlib import Path
import pandas as pd
import json



def load_data(train_filepath=None, test_filepath=None):
    """
    Carrega dados de treino e teste de diferentes formatos (.npz, .csv, .json, .xlsx, etc.).

    Args:
        train_filepath (str): Caminho do arquivo de treinamento.
        test_filepath (str): Caminho do arquivo de teste.

    Returns:
        tuple: (X_train, y_train, X_test, y_test)
    """

    def _load_file(filepath):
        """Carrega um único arquivo com base na extensão."""
        if filepath is None:
            return None, None

        file_extension = Path(filepath).suffix.lower()

        if file_extension == ".npz":
            data = np.load(filepath)
            return data["features"], data["labels"]
        elif file_extension == ".csv":
            df = pd.read_csv(filepath)
            return df.iloc[:, :-1].values, df.iloc[:, -1].values
        elif file_extension == ".xlsx":
            df = pd.read_excel(filepath)
            return df.iloc[:, :-1].values, df.iloc[:, -1].values
        elif file_extension == ".json":
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
            return np.array(data["features"]), np.array(data["labels"])
        elif file_extension == ".parquet":
            df = pd.read_parquet(filepath)
            return df.iloc[:, :-1].values, df.iloc[:, -1].values
        else:
            raise ValueError(f"Formato de arquivo não suportado: {file_extension}")

    X_train, y_train = _load_file(train_filepath)
    X_test, y_test = _load_file(test_filepath)

    if X_train is None or y_train is None:
        raise ValueError("Arquivo de treino não foi carregado corretamente.")
    if X_test is None or y_test is None:
        raise ValueError("Arquivo de teste não foi carregado corretamente.")

    return X_train, y_train, X_test, y_test


def load_data2(filepath=None):
          """
          Carrega dados de diferentes formatos, como .npz, .csv, .json, .xlsx, etc.

          Args:
              filepath (str): Caminho do arquivo. Se None, carrega o dataset Iris.

          Returns:
              tuple: (features, labels) onde features são as features e labels são os rótulos.
          """
          if filepath is None:
              dataset = load_iris()
              return dataset.data, dataset.target

          file_extension = Path(filepath).suffix.lower()

          if file_extension == ".npz":
              data = np.load(filepath)
              X, y = data["features"], data["labels"]
          elif file_extension == ".csv":
              df = pd.read_csv(filepath)
              X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
          elif file_extension == ".xlsx":
              df = pd.read_excel(filepath)
              X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
          elif file_extension == ".json":
              with open(filepath, "r", encoding="utf-8") as f:
                  data = json.load(f)
              X, y = np.array(data["features"]), np.array(data["labels"])
          elif file_extension == ".parquet":
              df = pd.read_parquet(filepath)
              X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
          else:
              raise ValueError(f"Formato de arquivo não suportado: {file_extension}")

          return X, y

class CrossValidation:
    """
    Classe para realizar validação cruzada e avaliar modelos.
    """
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.random_state = random_state
        self.kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    def evaluate_model(self, model, X, y):
        """
        Avalia um modelo usando validação cruzada.
        Args:
            model: Instância do modelo a ser avaliado.
            X (np.ndarray): Dados de entrada.
            y (np.ndarray): Rótulos.
        Returns:
            dict: Dicionário com as métricas de avaliação (acurácia, precisão, recall, F1-score, ROC-AUC).
        """
        accuracies = []
        precisions = []
        recalls = []
        f1_scores = []
        roc_aucs = []

        for train_index, val_index in self.kf.split(X, y):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            y_pred_proba = model.predict_proba(X_val) if hasattr(model, "predict_proba") else None

            # Calcular métricas
            accuracies.append(accuracy_score(y_val, y_pred))
            precisions.append(precision_score(y_val, y_pred, average="weighted"))
            recalls.append(recall_score(y_val, y_pred, average="weighted"))
            f1_scores.append(f1_score(y_val, y_pred, average="weighted"))

            # ROC-AUC (apenas para modelos que suportam predict_proba)
        if y_pred_proba is not None:
            # Verifica se é binário ou multiclasse
            if y_pred_proba.shape[1] == 2:  # Problema binário
                roc_aucs.append(roc_auc_score(y_val, y_pred_proba[:, 1]))
            else:  # Problema multiclasse
                y_val_bin = label_binarize(y_val, classes=np.unique(y))
                roc_aucs.append(roc_auc_score(y_val_bin, y_pred_proba, multi_class="ovr"))

        # Retornar métricas médias
        metrics = {
            "accuracy": np.mean(accuracies),
            "precision": np.mean(precisions),
            "recall": np.mean(recalls),
            "f1_score": np.mean(f1_scores),
            "roc_auc": np.mean(roc_aucs) if roc_aucs else None,
        }
        return metrics


class ModelTrainer:
    """
    Classe para treinar e avaliar modelos com base nos melhores hiperparâmetros.
    """
    def __init__(self, model, best_params):
        self.model = model
        self.best_params = best_params

    def train_final_model(self, X_train, y_train):
        """
        Treina o modelo final com os melhores hiperparâmetros.
        Args:
            X_train (np.ndarray): Dados de treinamento.
            y_train (np.ndarray): Rótulos de treinamento.
        """
        self.model.set_params(**self.best_params)
        self.model.fit(X_train, y_train)

    def evaluate_final_model(self, X_test, y_test):
        """
        Avalia o modelo final no conjunto de teste.
        Args:
            X_test (np.ndarray): Dados de teste.
            y_test (np.ndarray): Rótulos de teste.
        Returns:
            dict: Dicionário com as métricas de avaliação (acurácia, precisão, recall, F1-score, ROC-AUC).
        """
        y_pred = self.model.predict(X_test)
        y_pred_proba = self.model.predict_proba(X_test) if hasattr(self.model, "predict_proba") else None

        # ROC-AUC (apenas para modelos que suportam predict_proba)
        if y_pred_proba is not None:
            # Verifica se é binário ou multiclasse
            if y_pred_proba.shape[1] == 2:  # Problema binário
                roc_aucs = roc_auc_score(y_test, y_pred_proba[:, 1])
            else:  # Problema multiclasse
                #y_val_bin = label_binarize(y_test, classes=np.unique(y))
                #roc_aucs = roc_auc_score(y_val_bin, y_pred_proba, multi_class="ovr")
                roc_aucs = None

        # Calcular métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, average="weighted"),
            "recall": recall_score(y_test, y_pred, average="weighted"),
            "f1_score": f1_score(y_test, y_pred, average="weighted"),
            "roc_auc": roc_aucs if y_pred_proba is not None else None,
        }
        return metrics


class Experiment:
    """
    Classe para executar o experimento completo.
    """
    def __init__(self, train_filepath, test_filepath,random_state=42):
        self.X_train_val, self.y_train_val, self.X_test, self.y_test = load_data(train_filepath, test_filepath)
        self.random_state = random_state


    def _split_data(self):
        """
        Divide o dataset em treino/validação e teste.
        Returns:
            tuple: (X_train_val, X_test, y_train_val, y_test)
        """
        return train_test_split(self.X, self.y, test_size=self.test_size, random_state=self.random_state)


    def run_experiment(self):
        """
        Executa o experimento completo.
        """
        # Configurações de hiperparâmetros
        svm_params = {'C': [0.01, 0.1, 1, 10, 100]}
        knn_params = {'n_neighbors': [1, 3, 5, 7, 9]}

        # Validação cruzada para SVM
        cv = CrossValidation()
        svm_results = {}
        for C in svm_params['C']:
            model = LinearSVC(random_state=self.random_state)
            metrics = cv.evaluate_model(model.set_params(C=C), self.X_train_val, self.y_train_val)
            svm_results[C] = metrics
            print(f"SVM (C={C}):")
            print(f"  Acurácia: {metrics['accuracy']:.4f}")
            print(f"  Precisão: {metrics['precision']:.4f}")
            print(f"  Recall: {metrics['recall']:.4f}")
            print(f"  F1-score: {metrics['f1_score']:.4f}")
            print(f"  ROC-AUC: {metrics['roc_auc'] if metrics['roc_auc'] is not None else 'N/A'}")

        # Validação cruzada para KNN
        knn_results = {}
        for k in knn_params['n_neighbors']:
            model = KNeighborsClassifier()
            metrics = cv.evaluate_model(model.set_params(n_neighbors=k), self.X_train_val, self.y_train_val)
            knn_results[k] = metrics
            print(f"\nKNN (k={k}):")
            print(f"  Acurácia: {metrics['accuracy']:.4f}")
            print(f"  Precisão: {metrics['precision']:.4f}")
            print(f"  Recall: {metrics['recall']:.4f}")
            print(f"  F1-score: {metrics['f1_score']:.4f}")
            print(f"  ROC-AUC: {metrics['roc_auc'] if metrics['roc_auc'] is not None else 'N/A'}")

        # Selecionar os melhores hiperparâmetros
        best_C = max(svm_results, key=lambda x: svm_results[x]['accuracy'])
        best_k = max(knn_results, key=lambda x: knn_results[x]['accuracy'])
        print(f"\nMelhor C para SVM: {best_C}")
        print(f"Melhor k para KNN: {best_k}")

        # Treinar e avaliar os modelos finais
        svm_trainer = ModelTrainer(LinearSVC(random_state=self.random_state), {'C': best_C})
        svm_trainer.train_final_model(self.X_train_val, self.y_train_val)
        svm_metrics = svm_trainer.evaluate_final_model(self.X_test, self.y_test)
        print("\nMétricas finais - SVM:")
        print(f"  Acurácia: {svm_metrics['accuracy']:.4f}")
        print(f"  Precisão: {svm_metrics['precision']:.4f}")
        print(f"  Recall: {svm_metrics['recall']:.4f}")
        print(f"  F1-score: {svm_metrics['f1_score']:.4f}")
        print(f"  ROC-AUC: {svm_metrics['roc_auc'] if svm_metrics['roc_auc'] is not None else 'N/A'}")

        knn_trainer = ModelTrainer(KNeighborsClassifier(), {'n_neighbors': best_k})
        knn_trainer.train_final_model(self.X_train_val, self.y_train_val)
        knn_metrics = knn_trainer.evaluate_final_model(self.X_test, self.y_test)
        print("\nMétricas finais - KNN:")
        print(f"  Acurácia: {knn_metrics['accuracy']:.4f}")
        print(f"  Precisão: {knn_metrics['precision']:.4f}")
        print(f"  Recall: {knn_metrics['recall']:.4f}")
        print(f"  F1-score: {knn_metrics['f1_score']:.4f}")
        print(f"  ROC-AUC: {knn_metrics['roc_auc'] if knn_metrics['roc_auc'] is not None else 'N/A'}")


# Executar o experimento
if __name__ == "__main__":
    #iris = load_iris()
    train_file = "/content/drive/My Drive/Mestrado 2024/Projetos/VIT and CNN/Resultados/Dados compilados/google_vit_large_patch32_384.npz"  # caminho do seu arquivo
    test_file = "/content/drive/My Drive/Mestrado 2024/Projetos/VIT and CNN/Resultados/Dados compilados/google_vit_large_patch32_384_test.npz"  # caminho do seu arquivo

    experiment = Experiment(train_file, test_file)

    experiment.run_experiment()

SVM (C=0.01):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: N/A
SVM (C=0.1):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: N/A
SVM (C=1):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: N/A
SVM (C=10):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: N/A
SVM (C=100):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: N/A

KNN (k=1):
  Acurácia: 0.9983
  Precisão: 0.9983
  Recall: 0.9983
  F1-score: 0.9983
  ROC-AUC: 0.9946236559139785

KNN (k=3):
  Acurácia: 0.9979
  Precisão: 0.9979
  Recall: 0.9979
  F1-score: 0.9979
  ROC-AUC: 0.9999861076380206

KNN (k=5):
  Acurácia: 0.9967
  Precisão: 0.9967
  Recall: 0.9967
  F1-score: 0.9967
  ROC-AUC: 0.9999305381901031

KNN (k=7):
  Acurácia: 0.9946
  Precisão: 0.9946
  Recall: 0.9946
  F1-score: 0.9945
  ROC-AUC: 0.9999861076380205

KNN (k=9):
  Acurácia: 0.99

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
